<a href="https://colab.research.google.com/github/OmdenaAI/SudanChapter_AnalyzeHealthcareAccessibility/blob/main/04_Data_analysis/Geographic/Omedena_geographic_segment_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ast
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

import gdown

# رابط الملف على Google Drive
file_url = "https://drive.google.com/uc?id=1JXR_YZJEpIE5Eb0YbJYeo2cxmfMHbRBR"

# مسار التنزيل
output = "geographic_merged.csv"

# تحميل الملف
gdown.download(file_url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1JXR_YZJEpIE5Eb0YbJYeo2cxmfMHbRBR
From (redirected): https://drive.google.com/uc?id=1JXR_YZJEpIE5Eb0YbJYeo2cxmfMHbRBR&confirm=t&uuid=312857c4-fc76-4bee-941d-85134f6c10c5
To: /content/geographic_merged.csv
100%|██████████| 66.4M/66.4M [00:01<00:00, 34.2MB/s]


'geographic_merged.csv'

In [4]:
# قراءة الملف
file_path = "/content/geographic_merged.csv"
geo = pd.read_csv(file_path)

In [5]:
geo.head()

,GID_L0,GID_L1,GID_L2,GID_L3,COUNTRY,NAME_L1,NAME_L2,NAME_L3,GEOMETRY_L3,NAME_ALTERNATIVES_L1,...,COORDINATES_L1,COORDINATES_L2,COORDINATES_L3,CENTROID_COORDINATES_L1,CENTROID_COORDINATES_L2,CENTROID_COORDINATES_L3,SENTINEL_2_NDVI_L3,SENTINEL_2_NDWI_L3,WEATHER_WEEKLY_L1,MAJOR_CITIES_L1
0,SDN,SDN.1_1,SDN.1.1_1,SDN.1.1.1_1,Sudan,AlJazirah,AlKamlin,ElKamlin,"MULTIPOLYGON (((32.9784 15.1733, 33.0415 15.24...",Gezira,...,"[[[(32.7365, 13.7191), (32.7335, 13.749), (32....","[[[(33.1141, 14.8749), (32.9916, 14.9108), (32...","[[[(32.9784, 15.1733), (33.0415, 15.2452), (33...","(14.582373531656884, 33.345453919938265)","(15.097663580487422, 32.96771659913109)","(15.083964443779728, 33.11378956689792)","{'data': [{'interval_from': '2015-10-20', 'int...","{'data': [{'interval_from': '2015-10-20', 'int...","{'2022-01-01': {'cloud_cover': 65.0, 'humidity...",NaN
1,SDN,SDN.1_1,SDN.1.1_1,SDN.1.1.2_1,Sudan,AlJazirah,AlKamlin,ElMasid,"MULTIPOLYGON (((33.0415 15.2452, 32.9784 15.17...",Gezira,...,"[[[(32.7365, 13.7191), (32.7335, 13.749), (32....","[[[(33.1141, 14.8749), (32.9916, 14.9108), (32...","[[[(33.0415, 15.2452), (32.9784, 15.1733), (32...","(14.582373531656884, 33.345453919938265)","(15.097663580487422, 32.96771659913109)","(15.191525747215783, 32.86662373624138)","{'data': [{'interval_from': '2015-10-20', 'int...","{'data': [{'interval_from': '2015-10-20', 'int...","{'2022-01-01': {'cloud_cover': 65.0, 'humidity...",NaN
2,SDN,SDN.1_1,SDN.1.1_1,SDN.1.1.3_1,Sudan,AlJazirah,AlKamlin,ElMiEiliq,"MULTIPOLYGON (((32.9784 15.1733, 33.2035 14.88...",Gezira,...,"[[[(32.7365, 13.7191), (32.7335, 13.749), (32....","[[[(33.1141, 14.8749), (32.9916, 14.9108), (32...","[[[(32.9784, 15.1733), (33.2035, 14.8855), (33...","(14.582373531656884, 33.345453919938265)","(15.097663580487422, 32.96771659913109)","(15.00222006157765, 33.039896335888756)","{'data': [{'interval_from': '2015-10-20', 'int...","{'data': [{'interval_from': '2015-10-20', 'int...","{'2022-01-01': {'cloud_cover': 65.0, 'humidity...",NaN
3,SDN,SDN.1_1,SDN.1.1_1,SDN.1.1.4_1,Sudan,AlJazirah,AlKamlin,EsSideira,"MULTIPOLYGON (((32.5843 15.2263, 32.6041 15.22...",Gezira,...,"[[[(32.7365, 13.7191), (32.7335, 13.749), (32....","[[[(33.1141, 14.8749), (32.9916, 14.9108), (32...","[[[(32.5843, 15.2263), (32.6041, 15.2208), (32...","(14.582373531656884, 33.345453919938265)","(15.097663580487422, 32.96771659913109)","(15.170447598810247, 32.70681194553072)","{'data': [{'interval_from': '2015-10-20', 'int...","{'data': [{'interval_from': '2015-10-20', 'int...","{'2022-01-01': {'cloud_cover': 65.0, 'humidity...",NaN
4,SDN,SDN.1_1,SDN.1.2_1,SDN.1.2.1_1,Sudan,AlJazirah,AlMahagil,AlAzazi,"MULTIPOLYGON (((32.6635 14.3983, 32.6733 14.44...",Gezira,...,"[[[(32.7365, 13.7191), (32.7335, 13.749), (32....","[[[(32.5719, 13.9301), (32.5852, 13.9903), (32...","[[[(32.6635, 14.3983), (32.6733, 14.4499), (32...","(14.582373531656884, 33.345453919938265)","(14.33018702124788, 32.77158872373312)","(14.471606486513101, 32.74934652641849)","{'data': [{'interval_from': '2015-10-20', 'int...","{'data': [{'interval_from': '2015-10-20', 'int...","{'2022-01-01': {'cloud_cover': 65.0, 'humidity...",NaN


In [6]:
geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   GID_L0                   237 non-null    object
 1   GID_L1                   237 non-null    object
 2   GID_L2                   237 non-null    object
 3   GID_L3                   237 non-null    object
 4   COUNTRY                  237 non-null    object
 5   NAME_L1                  237 non-null    object
 6   NAME_L2                  237 non-null    object
 7   NAME_L3                  237 non-null    object
 8   GEOMETRY_L3              237 non-null    object
 9   NAME_ALTERNATIVES_L1     232 non-null    object
 10  TYPE_L1                  237 non-null    object
 11  HASC_L1                  237 non-null    object
 12  ISO_L1                   135 non-null    object
 13  GEOMETRY_L1              237 non-null    object
 14  TYPE_L2                  237 non-null    o

In [7]:
geo.nunique()

,0
GID_L0,1
GID_L1,18
GID_L2,80
GID_L3,237
COUNTRY,1
NAME_L1,18
NAME_L2,79
NAME_L3,233
GEOMETRY_L3,237
NAME_ALTERNATIVES_L1,17


In [8]:
geo["SENTINEL_2_NDVI_L3"] = geo["SENTINEL_2_NDVI_L3"].apply(lambda x: ast.literal_eval(x))
geo["SENTINEL_2_NDWI_L3"] = geo["SENTINEL_2_NDWI_L3"].apply(lambda x: ast.literal_eval(x))
geo["WEATHER_WEEKLY_L1"] = geo["WEATHER_WEEKLY_L1"].apply(lambda x: ast.literal_eval(x))

In [9]:
def flatten_sentinel_data(df, column, region_col, geom_col):
    records = []
    for idx, row in df.iterrows():
        if isinstance(row[column], dict):
            for key, values in row[column].items():
                for item in values:
                    records.append({
                        'date': pd.to_datetime(item['interval_from']),
                        'value': item['ndvi_mean'],
                        'region': row[region_col],
                        'geometry': row[geom_col]
                    })
    return pd.DataFrame(records)

In [10]:
geo.columns

Index(['GID_L0', 'GID_L1', 'GID_L2', 'GID_L3', 'COUNTRY', 'NAME_L1', 'NAME_L2',
       'NAME_L3', 'GEOMETRY_L3', 'NAME_ALTERNATIVES_L1', 'TYPE_L1', 'HASC_L1',
       'ISO_L1', 'GEOMETRY_L1', 'TYPE_L2', 'GEOMETRY_L2', 'COORDINATES_L1',
       'COORDINATES_L2', 'COORDINATES_L3', 'CENTROID_COORDINATES_L1',
       'CENTROID_COORDINATES_L2', 'CENTROID_COORDINATES_L3',
       'SENTINEL_2_NDVI_L3', 'SENTINEL_2_NDWI_L3', 'WEATHER_WEEKLY_L1',
       'MAJOR_CITIES_L1'],
      dtype='object')

In [11]:
ndvi_data = flatten_sentinel_data(geo, 'SENTINEL_2_NDVI_L3', 'NAME_L3', 'GEOMETRY_L3')
ndwi_data = flatten_sentinel_data(geo, 'SENTINEL_2_NDWI_L3', 'NAME_L3', 'GEOMETRY_L3')

In [12]:
ndvi_data.columns

Index(['date', 'value', 'region', 'geometry'], dtype='object')

In [13]:
ndvi_data.head()

,date,value,region,geometry
0,2015-10-20,0.005310,ElKamlin,"MULTIPOLYGON (((32.9784 15.1733, 33.0415 15.24..."
1,2015-11-29,0.017227,ElKamlin,"MULTIPOLYGON (((32.9784 15.1733, 33.0415 15.24..."
2,2015-12-09,0.058856,ElKamlin,"MULTIPOLYGON (((32.9784 15.1733, 33.0415 15.24..."
3,2015-12-19,0.030519,ElKamlin,"MULTIPOLYGON (((32.9784 15.1733, 33.0415 15.24..."
4,2015-12-29,0.084125,ElKamlin,"MULTIPOLYGON (((32.9784 15.1733, 33.0415 15.24..."


In [14]:
# Concating features
geo_data=pd.concat([geo,ndvi_data ,ndwi_data], axis=1)
geo_data.head()

,GID_L0,GID_L1,GID_L2,GID_L3,COUNTRY,NAME_L1,NAME_L2,NAME_L3,GEOMETRY_L3,NAME_ALTERNATIVES_L1,...,WEATHER_WEEKLY_L1,MAJOR_CITIES_L1,date,value,region,geometry,date,value,region,geometry
0,SDN,SDN.1_1,SDN.1.1_1,SDN.1.1.1_1,Sudan,AlJazirah,AlKamlin,ElKamlin,"MULTIPOLYGON (((32.9784 15.1733, 33.0415 15.24...",Gezira,...,"{'2022-01-01': {'cloud_cover': 65.0, 'humidity...",NaN,2015-10-20,0.005310,ElKamlin,"MULTIPOLYGON (((32.9784 15.1733, 33.0415 15.24...",2015-10-20,-0.011545,ElKamlin,"MULTIPOLYGON (((32.9784 15.1733, 33.0415 15.24..."
1,SDN,SDN.1_1,SDN.1.1_1,SDN.1.1.2_1,Sudan,AlJazirah,AlKamlin,ElMasid,"MULTIPOLYGON (((33.0415 15.2452, 32.9784 15.17...",Gezira,...,"{'2022-01-01': {'cloud_cover': 65.0, 'humidity...",NaN,2015-11-29,0.017227,ElKamlin,"MULTIPOLYGON (((32.9784 15.1733, 33.0415 15.24...",2015-11-29,-0.040840,ElKamlin,"MULTIPOLYGON (((32.9784 15.1733, 33.0415 15.24..."
2,SDN,SDN.1_1,SDN.1.1_1,SDN.1.1.3_1,Sudan,AlJazirah,AlKamlin,ElMiEiliq,"MULTIPOLYGON (((32.9784 15.1733, 33.2035 14.88...",Gezira,...,"{'2022-01-01': {'cloud_cover': 65.0, 'humidity...",NaN,2015-12-09,0.058856,ElKamlin,"MULTIPOLYGON (((32.9784 15.1733, 33.0415 15.24...",2015-12-09,-0.094806,ElKamlin,"MULTIPOLYGON (((32.9784 15.1733, 33.0415 15.24..."
3,SDN,SDN.1_1,SDN.1.1_1,SDN.1.1.4_1,Sudan,AlJazirah,AlKamlin,EsSideira,"MULTIPOLYGON (((32.5843 15.2263, 32.6041 15.22...",Gezira,...,"{'2022-01-01': {'cloud_cover': 65.0, 'humidity...",NaN,2015-12-19,0.030519,ElKamlin,"MULTIPOLYGON (((32.9784 15.1733, 33.0415 15.24...",2015-12-19,0.015846,ElKamlin,"MULTIPOLYGON (((32.9784 15.1733, 33.0415 15.24..."
4,SDN,SDN.1_1,SDN.1.2_1,SDN.1.2.1_1,Sudan,AlJazirah,AlMahagil,AlAzazi,"MULTIPOLYGON (((32.6635 14.3983, 32.6733 14.44...",Gezira,...,"{'2022-01-01': {'cloud_cover': 65.0, 'humidity...",NaN,2015-12-29,0.084125,ElKamlin,"MULTIPOLYGON (((32.9784 15.1733, 33.0415 15.24...",2015-12-29,-0.074044,ElKamlin,"MULTIPOLYGON (((32.9784 15.1733, 33.0415 15.24..."


In [ ]:

num=geo["COORDINATES_L1"].replace(" ", "").split(",")

# تحويل الأرقام إلى DataFrame
df = pd.DataFrame([num], columns=["COORDINATES_L1.1", "COORDINATES_L1.2"])

# طباعة الجدول
print(df)

AttributeError: 'Series' object has no attribute 'split'

In [ ]:

num = geo["COORDINATES_L1"].str.replace(" ", "").str.split(",", n=1)  # Limit split to 1

# Extract coordinates using apply
df = pd.DataFrame(num.tolist(), columns=["COORDINATES_L1.1", "COORDINATES_L1.2"])

# Assuming COORDINATES_L1 contains nested lists, extract lat and lon:
df["COORDINATES_L1.1"] = df["COORDINATES_L1.1"].apply(lambda x: ast.literal_eval(x[0])[0][0][0] if isinstance(x,list) and len(x) > 0 else None)
df["COORDINATES_L1.2"] = df["COORDINATES_L1.2"].apply(lambda x: ast.literal_eval(x[0])[0][0][1] if isinstance(x,list) and len(x) > 0 else None)
print(df)

ValueError: 2 columns passed, passed data had 4 columns